In [1]:
!pip install accelerate
!pip install transformers==4.45.2
!pip install -U bitsandbytes
!pip install datasets
!pip install rouge-score
!pip install pymorphy3
!pip install peft
!pip install flash_attn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
!git clone https://huggingface.co/datasets/nyu-mll/glue
!git clone https://github.com/RefalMachine/llmtf_open
%cd llmtf_open
!wget https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/train.jsonl

Cloning into 'glue'...
remote: Enumerating objects: 153, done.
remote: Total 153 (delta 0), reused 0 (delta 0), pack-reused 153 (from 1)
Receiving objects: 100% (153/153), 43.63 KiB | 14.54 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Filtering content: 100% (34/34), 154.73 MiB | 66.71 MiB/s, done.
Cloning into 'llmtf_open'...
remote: Enumerating objects: 655, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 655 (delta 161), reused 141 (delta 73), pack-reused 420 (from 1)
Receiving objects: 100% (655/655), 2.32 MiB | 3.50 MiB/s, done.
Resolving deltas: 100% (446/446), done.
/content/llmtf_open
--2025-04-17 23:15:19--  https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/master/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:4

In [1]:
%cd llmtf_open/

/content/llmtf_open


In [4]:
%cd ./llmtf_open/


[Errno 2] No such file or directory: './llmtf_open/'
/content/llmtf_open


In [2]:
from llmtf.model import HFModel
# from datasets import load_dataset, Dataset
from typing import Dict, List, Tuple
from llmtf.metrics import mean
from transformers import AutoTokenizer, AutoModelForCausalLM
from llmtf.base import SimpleFewShotHFTask
from sklearn.metrics import matthews_corrcoef



In [3]:
class GlueColaTask(SimpleFewShotHFTask):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.method = 'calculate_tokens_proba'
        self._max_new_tokens = 1

    @classmethod
    def name(cls):
        return 'glue/cola'

    @property
    def choices(self):
        return ["0", "1"]

    def aggregation(self) -> Dict:
        return {"acc": mean, "mcc": lambda data: matthews_corrcoef([d[0] for d in data],[d[1] for d in data])}

    def dataset_args(self) -> Dict:
        return {'path': '../glue/cola'}


    def evaluate(self, sample, y_pred) -> Dict:
        y_true = str(sample['label'])
        y_pred = sorted([pair for pair in y_pred.items()], key=lambda x: -x[1])[0][0]
        return {"acc": y_true == y_pred, "mcc": [y_true, y_pred]}

    def test_split_name(self) -> str:
        return 'validation'

    def prompt_split_name(self) -> str:
        return 'train'

    def create_messages(self, sample, with_answer=False) -> List[Dict]:

        messages = []

        instruction_user = "Your task is to determine the acceptability of the text for the English language in terms of syntax, morphology and semantics. The answer should be one number: 0 or 1, where 0 means the sentence is not acceptable from the point of view of the English language, 1 means it is acceptable.\nText:{sentence}"
        instruction_bot = "Answer:"

        messages.append({'role': 'user', 'content': instruction_user.format(**sample)})
        messages.append({'role': 'bot', 'content': instruction_bot})

        return messages

class RuColaCustomTask(SimpleFewShotHFTask):
    RUCOLA_HF_PATH = 'RussianNLP/rucola'
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.method = 'calculate_tokens_proba'
        self._max_new_tokens = 1

    def evaluate(self, sample, y_pred) -> Dict:
        y_true = str(sample['label'])
        y_pred = sorted([pair for pair in y_pred.items()], key=lambda x: -x[1])[0][0]
        return {"acc": y_true == y_pred, "mcc": [y_true, y_pred]}

    @classmethod
    def name(cls):
        return 'russiannlp/rucola_custom'

    @property
    def choices(self):
        return ["0", "1"]

    def aggregation(self) -> Dict:
        return {"acc": mean, "mcc": lambda data: matthews_corrcoef([d[0] for d in data], [d[1] for d in data])}

    def dataset_args(self) -> Dict:
        return {'path': self.RUCOLA_HF_PATH}

    def test_split_name(self) -> str:
        return 'validation'

    def prompt_split_name(self) -> str:
        return 'train'

    def create_messages(self, sample, with_answer):
        messages = []
        instruction_user = 'Твоя задача определить приемлемость текста для русского языка с точки зрения синтаксиса, морфологии и семантики. Ответом должно служить одно число: 0 или 1, где 0 - предложение не приемлемо с точки зрения русского языка, 1 - приемлемо.\nТекст: {sentence}'
        instruction_bot = 'Ответ: {label}'
        instruction_bot_incomplete = 'Ответ:'

        bot_content = instruction_bot.format(**sample) if with_answer else instruction_bot_incomplete

        messages.append({'role': 'user', 'content': instruction_user.format(**sample)})
        messages.append({'role': 'bot', 'content': bot_content})

        return messages

    def prompt_dataset_start_idx(self) -> int:
        # в ближайших индексах после 29 сбалансировано по меткам классов, вот поэтому
        return 29

In [4]:
task_ru = RuColaCustomTask()
task_en = GlueColaTask()
model_name = 'Qwen/Qwen2.5-1.5B-Instruct'

In [5]:
model = HFModel(
    device_map="cuda", # В случае нескольких GPU нужно указать конкретную, cuda:0, например
    attn_implementation="sdpa", #attn_implementation='flash_attention_2' для GPU, которые поддерживают
)
model.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


FORCES SYSTEM PROMPT AT START=<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>



100%|██████████| 151665/151665 [00:00<00:00, 506321.46it/s]
INFO: 2025-04-17 23:19:19,122: llmtf.base.hfmodel: Set eos_token_id in generation_config to [151645]
INFO:llmtf.base.hfmodel:Set eos_token_id in generation_config to [151645]
INFO: 2025-04-17 23:19:19,125: llmtf.base.hfmodel: Model id: Qwen/Qwen2.5-1.5B-Instruct
INFO:llmtf.base.hfmodel:Model id: Qwen/Qwen2.5-1.5B-Instruct
INFO: 2025-04-17 23:19:19,126: llmtf.base.hfmodel: Leading space: False
INFO:llmtf.base.hfmodel:Leading space: False


In [ ]:
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     load_in_8bit=False,
#     device_map="cuda",
#     attn_implementation="sdpa", #attn_implementation='flash_attention_2' для GPU, которые поддерживают
# )
# model.from_pretrained(model_name)

In [ ]:
# model_name = '../Falcon3-3B-Instruct'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.chat_template
# print(tokenizer.apply_chat_template([{'role': 'user', 'content': 'Что такое LLM?'}], tokenize=False, add_generation_prompt=True))
# tokenizer.chat_template

In [6]:
from llmtf.evaluator import Evaluator
evaluator = Evaluator()

evaluator.evaluate_dataset(
    task=task_en,
    model=model,
    output_dir='../cola-qwen',
    max_len=4000,
    few_shot_count=0,
    generation_config=None, # will use model.generation_config by default
    batch_size=4,
    max_sample_per_dataset=200
)

INFO: 2025-04-17 23:19:22,492: llmtf.base.hfmodel: Updated generation_config.eos_token_id: [151645]
INFO:llmtf.base.hfmodel:Updated generation_config.eos_token_id: [151645]
INFO: 2025-04-17 23:19:22,496: llmtf.base.hfmodel: Updated generation_config.stop_strings: ['<|im_end|>']
INFO:llmtf.base.hfmodel:Updated generation_config.stop_strings: ['<|im_end|>']
100%|██████████| 200/200 [00:00<00:00, 2578.07it/s]
INFO: 2025-04-17 23:19:22,621: llmtf.base.glue/cola: Loading Dataset: 0.12s
INFO:llmtf.base.glue/cola:Loading Dataset: 0.12s
100%|██████████| 50/50 [00:23<00:00,  2.09it/s]
INFO: 2025-04-17 23:19:46,560: llmtf.base.glue/cola: Processing Dataset: 23.94s
INFO:llmtf.base.glue/cola:Processing Dataset: 23.94s
INFO: 2025-04-17 23:19:46,562: llmtf.base.glue/cola: Results for glue/cola:
INFO:llmtf.base.glue/cola:Results for glue/cola:
INFO: 2025-04-17 23:19:46,569: llmtf.base.glue/cola: {'acc': 0.745, 'mcc': np.float64(0.4985588860372235)}
INFO:llmtf.base.glue/cola:{'acc': 0.745, 'mcc': np.f

In [ ]:
!ls
!echo ""
!ls ./rucola-qwen/

In [ ]:
!cat ./rucola-qwen/russiannlp_rucola_custom_total.jsonl
